# Imperfection

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [45]:
from qmap import *
from IPython.html.widgets import *

In [113]:
def pl(g):
    markings = get_markings(100, 15)
#     markings[:, 0] = 1
    markings = gap_markings(markings, g)
#     markings = pepper_markings(markings, [0.5, 0.5, 1, 0, 0, 0, 0, 0, 0, 0], start=35)
    plot_mem(run(markings), markings)

In [114]:
interact(pl, g=(0, 20))

interactive(children=(IntSlider(value=10, description='g', max=20), Output()), _dom_classes=('widget-interact',))

<function __main__.pl>